<center><h2>ALTeGraD 2023<br>Lab Session 4: NLP Frameworks</h2> 07 / 11 / 2023<br> Dr. G. Shang, H. Abdine<br><br>


<b>Student name:</b> Balthazar Neveu

</center>
<font color='gray'>

In this lab you will learn how to use Fairseq and HuggingFace transformers - The most used libraries by researchers and developers  and finetune language models - to finetune a pretrained French language model ($RoBERTa_{small}^{fr}$) on the sentiment analysis dataset CLS_Books where each review is labeled as positive or negative and finetune a variant of BLOOM on a question/answer dataset.
</font>


In [ ]:
from pathlib import Path
def abspath(pth): return Path(pth).resolve()

<font color='gray'>

## Task 2: Preprocessing the dataset
Prepare the data for finetuning. To do so you have to:
1. Use the provided sentencepiece model to tokenize the text.
2. Binarize the data using `fairseq-preprocess` command

### Tokenizing the reviews

In this section we will tokenize the finetuning dataset using sentenpiece tokenizer. We have three splits in our datase: train valid and test sets.

In this task you have to use the trained sentencepiece tokenizer (RoBERTa_small_fr/sentencepiece.bpe.model) to tokenize 
- the input three files <b>train.review</b>, <b>valid.review</b> and <b>test.review</b> 
- and output the three files <b>train.spm.review</b>, <b>valid.spm.review</b> and <b>test.spm.review</b> containing the tokenized reviews.

Documentation: https://github.com/google/sentencepiece#readme
</font>

# Task 2.1 Tokenization
When we split the sentences into tokens here, we use a provided byte pair encoding model.

When training from scratch, we should start from scratch and tokenize a corpus from scratch...

but if we want to **re-use a pretrained model**, we need to follow its vocabulary definition.

In [ ]:
SPLITS=['train', 'test', 'valid']
TOKENIZER_MODEL_PATH = '../models/RoBERTa_small_fr/sentencepiece.bpe.model'
DATA_BOOKS = '../data/cls.books/'

In [ ]:
import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file=TOKENIZER_MODEL_PATH)

SENTS="review"

for split in SPLITS:
    with open(DATA_BOOKS+split+'.'+SENTS, 'r') as f:
        reviews = f.readlines()
        
        tokenized = [" ".join(s.encode(review, out_type=str)) for review in reviews]
        print(f"Original: {reviews[0][:80]}, \t Tokenized: {tokenized[0][:80]}")
        # tokenize the data using s.encode and a loop(check the documentation)

        # It should look something like this :
        #▁An ci enne ▁VS ▁Nouvelle ▁version ▁plus
    
    with open(DATA_BOOKS+split+'.spm' + "." + SENTS, 'w') as f:
        f.writelines("\n".join(tokenized)+'\n')

<font color='gray'>

## <b>Binarizing/Preprocessing the finetuning dataset</b>

In this section, you have to binarize the CLS_Books dataset using the <b>fairseq/fairseq_cli/preprocess.py</b> script:

1- Binarize the tokenized reviews and put the output in <b>data/cls-books-bin/input0</b>. 
> Note: Our pretrained model's embedding matrix contains only the embedding of the vocab listed in the dictionary <b>dict.txt</b>.
>
> You need to use the dictionary in the binarization of the text to transform the tokens into indices. Also note that we are using Encoder only architecture, so we only have source data.

2- Binarize the labels (train.label, valid.label and test.label files) and put the output in <b>data/cls-books-bin/label</b>.

Documentation: https://fairseq.readthedocs.io/en/latest/command_line_tools.html
</font>

## Task 2.1 Binarization

- `--only-source` : Only process the source language
- `srcdict` allows forcing the tokenization dictionary.
- `-s "spm.review"` for source language , `-t ""` for an undefined target language.

Note on compression
> - train.review `1102kb`
> - train.spm `1741kb`  (added extra characters to get ready for binarization)
> - binarized : `569kb` (file appears compressed, sentences transfers to GPU memory will be lighter).




In [ ]:
SRC_DICT = "../models/RoBERTa_small_fr/dict.txt"
DESTINATION_FOLDER = "../data/cls-books-bin/input0"
CORPUS_TRAIN, CORPUS_VALID, CORPUS_TEST = [f"../data/cls.books/{dataset_split}" for dataset_split in ['train', 'valid', 'test']]
# binarize the tokenized reviews
!(python ~/fairseq/fairseq_cli/preprocess.py \
            --only-source \
            --workers 8 \
            --srcdict "$SRC_DICT" \
            --destdir "$DESTINATION_FOLDER"\
            --trainpref "$CORPUS_TRAIN" \
            --validpref "$CORPUS_VALID" \
            --testpref "$CORPUS_TEST" \
            -s "spm.review" \
            -t "" \
)